[Reference](https://towardsdatascience.com/feature-selection-why-how-explained-part-2-352d9130c2e1)

In [1]:
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_features=20,
              n_informative=5, n_redundant=2, n_classes=4)

# 1. Wrapper Feature Selection

In [3]:
from sklearn.feature_selection import RFECV
import sklearn
svc = sklearn.svm.LinearSVC() #estimator
rfecv = RFECV(estimator=svc, scoring='accuracy')
rfecv.fit(X, y)
rfecv.support_ # check out which features were selected

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

array([ True,  True,  True, False, False, False, False, False,  True,
        True,  True, False,  True, False, False, False, False,  True,
       False, False])

# 2. Filtering Feature Selection

In [4]:
import numpy as np
def condition_number(corr):# corr: correlation matrix
    eigValues,eigVectors = np.linalg.eig(corr) 
    return abs(max(eigValues)/min(eigValues))
 # greadily kick out the worst feature
def eliminate_multicol(X, threshold = 100):#weak corr threshold
    corr = np.corrcoef(X.T) # get corr matrix
    if (condition_number(corr) < threshold or X.shape[1]<=1): 
        return X  
    # find the feature with highest corr coef(lower triangle)
    mostCorrelated = np.argmax([max(corr[i+1:,i]) for i in range(X.shape[1]-1)] )
    choose = [True]*X.shape[1]
    choose[mostCorrelated] = False # kick that feature out
    return eliminate_multicol(X[:,choose])

In [5]:
from sklearn.feature_selection import GenericUnivariateSelect as GUS, f_classif
X=eliminate_multicol(X) # remove multicollinearity
gus = GUS (f_classif, 'fwe', param=0.5) # keep features with P <0.5
X_new = gus.fit_transform(X, y) # remove irrelevant features

# 3. Embedded Feature Selection

In [6]:
from sklearn.linear_model import LassoCV
# import lasso with cross validation
lassoModel = LassoCV(cv=3).fit(X,y)